In [1]:
AKS_NAME="opiaks"
AML_WORKSPACE_NAME="opiworkspace-aml"
RESOURCE_GROUP="opiworkspace-common"

In [2]:
import os
from azureml.core import Workspace

# Connect to your AzureML Workspace
# Your AzureML Workspace name is equal to your IQStudio workspace name with -aml appended
# Example: If your IQStudio workspace name is iqsWorkspace, your AzureML workspace name will be iqsWorkspace-aml

ws = Workspace.get(name=AML_WORKSPACE_NAME,
#                subscription_id=os.getenv('AZ_SUBSCRIPTION_ID'),     # FOR VM RUN ONLY
                subscription_id='cf2a1e19-cf4a-40f9-8d04-6bc1ece52879', # FOR LOCAL RUN ONLY
               resource_group=RESOURCE_GROUP)

In [3]:
from azureml.core import Experiment
experiment_name = 'opiclaimsexp'
exp = Experiment(workspace=ws, name=experiment_name)

In [4]:
from azureml.core.model import Model
# register a model

In [6]:
 #out of a run record
#model = best_run.register_model(model_name='my_best_model', model_path='outputs/my_model.pkl')

# or, you can register a file or a folder of files as a model
model = Model.register(model_path="/Users/agupt38/Documents/idws-ml/model_objects", model_name="opi_classification2", workspace=ws)

Registering model opi_classification2


In [5]:
Model.get_model_path("opi_classification2",_workspace=ws)

'azureml-models/opi_classification2/3/model_objects'

In [6]:
from azureml.core import Run


In [15]:
#best_run = Run(experiment=exp, run_id="8700d6a0-0a71-4de4-a0ea-8403e06fdf4b")
#model = best_run.register_model(model_name='opi_classification5', model_path="regression_model.pkl")

In [12]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.model import Model, InferenceConfig

aks_target = AksCompute(ws,AKS_NAME)

deployment_config = AksWebservice.deploy_configuration(
    collect_model_data=True,
    cpu_cores=1,
    memory_gb=1,
    enable_app_insights=True,
    auth_enabled=True,
    namespace="models"
)

inference_config = InferenceConfig(
            entry_script="score_opi.py",
            conda_file="env_opi.yml",
            runtime="python")

In [13]:
model = ws.models['opi_classification2']
model

Model(workspace=Workspace.create(name='opiworkspace-aml', subscription_id='cf2a1e19-cf4a-40f9-8d04-6bc1ece52879', resource_group='opiworkspace-common'), name=opi_classification2, id=opi_classification2:3, version=3, tags={}, properties={})

In [14]:
service = Model.deploy(ws, 'opiclassificationwebservice2', [model], inference_config, deployment_config, aks_target, overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
2020-10-16T09:37:00,040275801+00:00 - rsyslog/run 
2020-10-16T09:37:00,040990910+00:00 - gunicorn/run 
2020-10-16T09:37:00,041970622+00:00 - iot-server/run 
2020-10-16T09:37:00,044023647+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_b7d3df66b23a7142da53eec3e1e19594/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b7d3df66b23a7142da53eec3e1e19594/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b7d3df66b23a7142da53eec3e1e19594/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b7d3df

In [ ]:
print(service.get_logs())